# Demo for Trading Backtesting

I have a previous project which is helpful to simply have something as a simple single stock backtester.

It is important to note that this does not include a universe of tickers or have shorting mechanics (*just has long positions and take profit/stop loss logic*)

In [21]:

import pandas as pd 
from enum import Enum
from datetime import datetime
import uuid 
import logging
import math

logging.basicConfig(
    filename="debug.log",
    filemode="w",
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.DEBUG
)


# prices = pd.read_csv("Nvidia2024-2014.csv")
prices = pd.read_csv("APPL.csv")

prices.rename(columns={"Close/Last": "Close"}, inplace=True)

usd_to_gbp_rate = 0.76

price_columns = ["Close", "Open", "High", "Low"]

prices["Date"] = pd.to_datetime(prices["Date"], format="%m/%d/%Y").dt.strftime("%Y-%m-%d")
prices = prices.sort_values(by="Date")

prices = prices.reset_index(drop=True)

prices.head()



for col in price_columns:
    prices[col] = prices[col].astype(str).replace({"\\$": ""}, regex=True).astype(float)

for col in price_columns:
    prices[col] = prices[col] * usd_to_gbp_rate
prices.head()

,Date,Close,Volume,Open,High,Low
0,2014-10-20,18.9544,302820160,18.679812,18.992400,18.6618
1,2014-10-21,19.4693,375146720,19.573800,19.573800,19.2413
2,2014-10-22,19.5681,272560040,19.539600,19.780900,19.4940
3,2014-10-23,19.9177,283040600,19.775200,19.959728,19.6897
4,2014-10-24,19.9918,187751520,19.984200,20.043100,19.8607


In [22]:
df = pd.DataFrame(prices)

df["AvgPrice"] = (df["Open"] + df["Close"]) / 2

df["AbsDiff"] = df["AvgPrice"].diff().abs()

cash = 1
trick = False
for index,row in df.iterrows():
    if(not trick):
        trick = True
        continue
    cash *= (float(row["AbsDiff"]) / float(row["AvgPrice"])) + 1
    
print("BOOK CASH LEVEL:", ("£{:,.2f}".format(cash)))
pnl = cash - 1
roi = (pnl / 1) * 100
print("RATE ON INVESTMENT: {:_.2f}%".format(roi))
print("ROUGH ANNUAL AVERAGE: {:_.2f}%".format(math.pow(roi,1/10)))

BOOK CASH LEVEL: £193,129,668,172.76
RATE ON INVESTMENT: 19_312_966_817_176.00%
ROUGH ANNUAL AVERAGE: 21.31%


## Initial framework for dummy trades

In [23]:
INITIAL_WALLET_CASH = 5_000_000
STANDARD_LOT = 100_000

class Side(Enum):
    BUY = "BUY"
    SELL = "SELL"
    SHORT = "SHORT"
    COVER = "COVER"
    
class Currency(Enum):
    GBP = "GBP"
    USD = "USD"
    
class Position:
    def __init__(self,
                 date:datetime,
                 size:int,
                 price,
                 currency:Currency,
                 side:Side,
                 is_short:bool = False):
        
        self._id = str(uuid.uuid4())
        self._date = date
        self._currency = currency
        self._price = price
        self._size = size
        self._side = side
        self._is_short = is_short
        
class Order:
    def __init__(self,
                 position:Position):
        self._id = str(uuid.uuid4())
        self._position = position
    
    def describe(self):
        return f"<Order id: {self._id} position: {self._position}>"

class Portfolio:
    
    def __init__(self):
        self.cash_wallet = INITIAL_WALLET_CASH  # wallet is in GBP
        self.current_positions = {}
        self.closed_positions = {}
        self.short_positions = {}
    
    def add_short_position(self, position):
        self.short_positions[position._id] = position
        self.add_to_wallet(self.calculate_short_credit(position))
    
    def cover_short_position(self, position):
        if position._id in self.short_positions:
            original_position = self.short_positions.pop(position._id)
            self.closed_positions[position._id] = position
            self.deduct_from_wallet(self.calculate_short_debit(original_position, position))
    
    def calculate_short_credit(self, position):
        return position._price * (position._size * STANDARD_LOT)
    
    def calculate_short_debit(self, original_position, cover_position):
        return cover_position._price * (original_position._size * STANDARD_LOT)
    
    def add_position(self,position):
        self.current_positions[position._id] = position
        self.deduct_from_wallet(self.calculate_deduction(position))
                
    def sell_position(self,position:Position):
        self.current_positions.pop(position._id)
        self.closed_positions[position._id] = position
    
    def get_wallet_value(self):
        return self.cash_wallet
    
    def deduct_from_wallet(self,amount):
        self.cash_wallet -= amount
        
    def add_to_wallet(self,amount):
        self.cash_wallet += amount
        
    def calculate_fees(self,position:Position):
        fee = position._size * 2 #broker fee is £2 per standard lot 
        logging.info(f"FEE CHARGED: £{fee}")
        return fee 
    
    def calculate_total_cost_in_gbp(self,position:Position):
        if(position._currency == Currency.GBP):
            return position._price * (position._size * STANDARD_LOT)
        # elif(position._currency == Currency.USD):
        #     return position._price * current_gbp_price  * (position._size * 100000) # x100000 for standard lot

    def calculate_deduction(self,position):
        return self.calculate_total_cost_in_gbp(position) + self.calculate_fees(position)
    

## Take Profit and Stop Loss mechanics

In [24]:
def check_take_profit(FIXED_LOT_SIZE, A_REALLY_BIG_NUMBER, orders, tp_prices, row, currentP, date):
    if (tp_prices != []):
        cTP = min(tp_prices)
            
        while(currentP>=cTP):
            sell_position = Position(date,FIXED_LOT_SIZE,row.Open,Currency.GBP,Side.SELL)
            sell_order = Order(sell_position)
            orders[sell_order._id] = sell_order
                
            tp_prices.pop(tp_prices.index(cTP))
            try:
                cTP = min(tp_prices)
            except ValueError:
                cTP = A_REALLY_BIG_NUMBER

def check_stop_loss(FIXED_LOT_SIZE, A_REALLY_SMALL_NUMBER, orders, sl_prices, row, currentP, date):
    if (sl_prices != []):
        cSL = max(sl_prices)

        while(currentP<=cSL):
            sell_position = Position(date,FIXED_LOT_SIZE,row.Open,Currency.GBP,Side.SELL)
            sell_order = Order(sell_position)
            orders[sell_order._id] = sell_order

            sl_prices.pop(sl_prices.index(cSL))
            try:
                cSL = max(sl_prices)

            except ValueError:
                cSL = A_REALLY_SMALL_NUMBER


## Setup indicators

Can add more strategies or remove ones that might not be impactful

In [25]:
def setup_indicators(rsi_level, bb_mult, macd_period, data):
     # calculate Bollinger Bands
    data["MA20"] = data["Close"].rolling(window=20).mean()
    data["MA20_std"] = data["Close"].rolling(window=20).std()
    data["Upper_band"] = data["MA20"] + (data["MA20_std"] * bb_mult)
    data["Lower_band"] = data["MA20"] - (data["MA20_std"] * bb_mult)
    
    # calculate RSI
    period = 14
    delta = data["Open"].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    AVG_Gain = up.rolling(window=period).mean()
    AVG_Loss = abs(down.rolling(window=period).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    data["RSI"] = RSI

     # calculate Moving Average Convergence Divergence (MACD)
    period_short = macd_period[0]
    period_long = macd_period[1]
    signal_period = 9
    data_length = len(data)
    data["EMA_short"] = data["Close"].ewm(span=period_short, adjust=False).mean()
    data["EMA_long"] = data["Close"].ewm(span=period_long, adjust=False).mean()
    data["MACD"] = data["EMA_short"] - data["EMA_long"]
    data["Signal"] = data["MACD"].ewm(span=signal_period, adjust=False).mean()

    # calculate Stochastic Oscillator
    period = 14
    data["L14"] = data["Low"].rolling(window=period).min()
    data["H14"] = data["High"].rolling(window=period).max()
    data["%K"] = (data["Close"] - data["L14"]) / (data["H14"] - data["L14"]) * 100
    data["%D"] = data["%K"].rolling(window=3).mean()
    return data


## Strategy function

edit the below cells to change the strategy, at the moment just use all the indicators to make a buy or sell order.
can add more "*special cases*" for potential trading opportunities missed

In [26]:
FIXED_LOT_SIZE = 10
A_REALLY_BIG_NUMBER = 100_000_000
A_REALLY_SMALL_NUMBER = 0.0000000000001

def check_to_sell(orders, row, current_price, date, rsi_level):
    if(row["Close"] > row["Upper_band"] or row["RSI"] > rsi_level or row["MACD"] < row["Signal"] and (row["%K"] > 80 and row["%D"] > 80 and row["%K"]<row["%D"])):
        sell_position = Position(date,FIXED_LOT_SIZE,current_price,Currency.GBP,Side.SELL)
        sell_order = Order(sell_position)
        orders[sell_order._id] = sell_order

def check_to_buy(orders, tp_prices, sl_prices, row, current_price, date, rsi_level):
    if((row["Close"] < row["Lower_band"] and row["RSI"] < rsi_level) or row["MACD"] > row["Signal"] and (row["%K"] < 20 and row["%D"] < 20 and row["%K"]<row["%D"])):
        buy_position = Position(date,FIXED_LOT_SIZE,current_price,Currency.GBP,Side.BUY)
        buy_order = Order(buy_position)
        orders[buy_order._id] = buy_order
        tp_prices.append(row["Upper_band"])
        sl_prices.append(row["Lower_band"])


In [27]:
def strategy(rsi_level, bb_mult, macd_period, data):
    
    orders = {}
    tp_prices=[]
    sl_prices=[]
    
    data = setup_indicators(rsi_level, bb_mult, macd_period, data)
    
    for index,row in data.iterrows():
        current_price = row["Open"]+row["Close"]/2

        date = datetime.strptime(str(row["Date"]),'%Y-%m-%d').date()
        
        check_take_profit(FIXED_LOT_SIZE, A_REALLY_BIG_NUMBER, orders, tp_prices, row, current_price, date)
        check_stop_loss(FIXED_LOT_SIZE, A_REALLY_SMALL_NUMBER, orders, sl_prices, row, current_price, date)

        
        check_to_buy(orders, tp_prices, sl_prices, row, current_price, date, rsi_level[0])
            

        check_to_sell(orders, row, current_price, date, rsi_level[1])

    return orders

## Backtest function

At the moment this just goes through all the orders made from the strategy function and simulates how profitiable it would be if the orders were made.

In [28]:
from collections import defaultdict


def backtest_strategy(order_queue):
    
    portfolio = Portfolio()
    side_counts = defaultdict(int)
    
    for index, order in order_queue.items():

        c_pnl = portfolio.cash_wallet - INITIAL_WALLET_CASH
        logging.info(f"Side is {order._position._side}")
        logging.info(f"Current P/L: {c_pnl}")

        # BUY orders
        if order._position._side == Side.BUY:
            side_counts["BUY"] += 1
            if portfolio.calculate_total_cost_in_gbp(order._position) <= portfolio.cash_wallet:
                portfolio.add_position(order._position)  # Deduct cash
                logging.info(f"BUY ORDER FILLED: (ID) {str(order._id).upper()}")
            else: 
                logging.info("INSUFFICIENT CASH FOR BUY ORDER")
        
        # SELL orders
        elif order._position._side == Side.SELL:
            side_counts["SELL"] += 1
            # Find the earliest matching BUY position (by size) to sell
            matching_position = next(
                (pos for pos in portfolio.current_positions.values()
                 if pos._side == Side.BUY and pos._size == order._position._size),
                None
            )
            
            if matching_position:
                sale_earning = order._position._price * (order._position._size * STANDARD_LOT)
                buy_cost = matching_position._price * (matching_position._size * STANDARD_LOT)
                portfolio.add_to_wallet(sale_earning)
                portfolio.sell_position(matching_position)             
                portfolio.add_to_wallet(buy_cost)
                fee = portfolio.calculate_fees(order._position)
                portfolio.deduct_from_wallet(fee)

                logging.info(f"SELL ORDER FILLED: (ID) {str(order._id).upper()}")
            else:
                logging.info("NO MATCHING BUY POSITION FOUND")
        
        # SHORT orders
        elif order._position._side == Side.SHORT:
            side_counts["SHORT"] += 1
            portfolio.add_short_position(order._position)
            logging.info(f"SHORT ORDER FILLED: (ID) {str(order._id).upper()}")

        # COVER orders
        elif order._position._side == Side.COVER:
            side_counts["COVER"] += 1
            matching_position = next(
                (pos for pos in portfolio.short_positions.values()
                 if pos._size == order._position._size),
                None
            )
            
            if matching_position:
                portfolio.cover_short_position(order._position)
                fee = portfolio.calculate_fees(order._position)
                portfolio.deduct_from_wallet(fee)
                logging.info(f"COVER ORDER FILLED: (ID) {str(order._id).upper()}")
            else:
                logging.info("NO MATCHING SHORT POSITION FOUND")
    
    # After all orders processed
    print("ALL ORDERS HAVE BEEN FILLED.")
    print("BOOK CASH LEVEL:", ("£{:,.2f}".format(portfolio.cash_wallet)))

    pnl = portfolio.cash_wallet - INITIAL_WALLET_CASH

    if pnl < 0:
        print("TOTAL LOSS:", ("£{:,.2f}".format(abs(pnl))))
    else:
        print("TOTAL PROFIT:", ("£{:,.2f}".format(pnl)))

    roi = (pnl / INITIAL_WALLET_CASH) * 100
    print("RATE ON INVESTMENT: {:_.2f}%".format(roi))
    print(f"BUY:{side_counts['BUY']}\nSELL:{side_counts['SELL']}\nSHORT:{side_counts['SELL']}\nCOVER:{side_counts['COVER']}")

    return pnl


## Function to test best params for strategy

In [9]:
import itertools



def test_strategy(strategy_func,data):
    params_for_testing = define_params()
    best_pnl = float("-inf")
    best_params = None

    for rsi_level, bb_mult, macd_period in itertools.product(*params_for_testing):
        orders = strategy_func(rsi_level, bb_mult, macd_period, data)
        pnl = backtest_strategy(orders)
        if pnl > best_pnl:
            best_pnl = pnl
            best_params = (rsi_level, bb_mult, macd_period)

    print("Best PnL:", best_pnl)
    roi = (best_pnl / INITIAL_WALLET_CASH) * 100
    print("RATE ON INVESTMENT: {:_.2f}%".format(roi))
    print(f"Optimal Parameters:\nRSI: low - {best_params[0][0]} high - {best_params[0][1]}\nBollinger Bands STD: {best_params[1]}\nMACD Time Period: short - {best_params[2][0]} long - {best_params[2][1]}")

def define_params():
    rsi_levels = [(low, high) for low in range(35, 37, 1) for high in range(60, 50, -1)]
    bollinger_mults = [x / 100 for x in range(100, 150, 10)]
    macd_periods = [(12,26)]

    num_combinations = len(rsi_levels) * len(bollinger_mults) * len(macd_periods)

    print(f"Number of RSI: {len(rsi_levels)}\nNumber of bollinger mults: {len(bollinger_mults)}\nNumber of MACD periods: {len(macd_periods)}\nNumber of combinations: {num_combinations}")

    return (rsi_levels,bollinger_mults,macd_periods)

## WARNING

**THIS IS A BRUTEFORCE APPROACH AND SO RESULTS IN A LOT OF COMBINATIONS**

You can refine the search space however, I saw this resulted in less trades being made netting 0% ROI. You can also change the ranges of the parameters being tested

Run the function below to check the number of combinations and to find what can be reduced

In [ ]:
define_params()

In [ ]:
test_strategy(strategy,prices)

## BEST STRATEGY?

RSI: low - 35 high - 65 <br>
Bollinger Bands STD: 1.5 <br>
MACD Time Period: short - 10 long - 24

In [17]:
orders = strategy((35,65),1.5,(10,24),prices)
backtest_strategy(orders)

ALL ORDERS HAVE BEEN FILLED.
BOOK CASH LEVEL: £5,000,000.00
TOTAL PROFIT: £0.00
RATE ON INVESTMENT: 0.00%
BUY:158
SELL:1145
SHORT:1145
COVER:0


0

In [ ]:
data = setup_indicators((30,65), 1.5, (12,26), prices)
data

Something to note: at the moment the orders price is just the price the order was put in for so we are assuming each order is fulfilled. Obviously in a real market if we put in an order to sell something for 1,000,000 it wouldn't be fulfilled. The strategy just enters orders using the average of the opening and the close prices, (again not very realistic so requires improvement)

In [29]:
def check_to_sell_with_short(orders, row, current_price, date, rsi_level):
    if(row["Close"] > row["MA20"]) and row["RSI"] > rsi_level and (row["MACD"] < row["Signal"] and (row["%K"] > 80 and row["%D"] > 80 and row["%K"]<row["%D"])):
        sell_position = Position(date,FIXED_LOT_SIZE,current_price,Currency.GBP,Side.SELL)
        sell_order = Order(sell_position)
        orders[sell_order._id] = sell_order

        short_position = Position(date, FIXED_LOT_SIZE, current_price, Currency.GBP, Side.SHORT, is_short=True)
        short_order = Order(short_position)
        orders[short_order._id] = short_order

def check_to_buy_with_cover(orders, tp_prices, sl_prices, row, current_price, date, rsi_level):
    if(row["Close"] < row["MA20"]) and row["RSI"] < rsi_level and (row["MACD"] > row["Signal"] and (row["%K"] < 20 and row["%D"] < 20 and row["%K"]<row["%D"])):
        buy_position = Position(date,FIXED_LOT_SIZE,current_price,Currency.GBP,Side.BUY)
        buy_order = Order(buy_position)
        orders[buy_order._id] = buy_order
        tp_prices.append(row["Upper_band"])
        sl_prices.append(row["Lower_band"])
        
        cover_position = Position(date, FIXED_LOT_SIZE, current_price, Currency.GBP, Side.COVER)
        cover_order = Order(cover_position)
        orders[cover_order._id] = cover_order
    


In [30]:
def strategy_with_short(rsi_level, bb_mult, macd_period, data):
    
    orders = {}
    tp_prices=[]
    sl_prices=[]
    
    data = setup_indicators(rsi_level, bb_mult, macd_period, data)
    
    for index,row in data.iterrows():
        current_price = row["Open"]+row["Close"]/2

        date = datetime.strptime(str(row["Date"]),"%Y-%m-%d").date()
        
        check_take_profit(FIXED_LOT_SIZE, A_REALLY_BIG_NUMBER, orders, tp_prices, row, current_price, date)
        check_stop_loss(FIXED_LOT_SIZE, A_REALLY_SMALL_NUMBER, orders, sl_prices, row, current_price, date)
        

        
        check_to_buy_with_cover(orders, tp_prices, sl_prices, row, current_price, date, rsi_level[0])
        check_to_sell_with_short(orders, row, current_price, date, rsi_level[1])

    return orders

In [31]:
orders = strategy_with_short((35,65),1.0,(12,26),prices)
backtest_strategy(orders)

ALL ORDERS HAVE BEEN FILLED.
BOOK CASH LEVEL: £1,179,577,068.00
TOTAL PROFIT: £1,174,577,068.00
RATE ON INVESTMENT: 23_491.54%
BUY:2
SELL:13
SHORT:13
COVER:2


1174577068.0

In [ ]:
orders = strategy_with_short((35,65),1.0,(12,26),prices)
backtest_strategy(orders)

In [ ]:
test_strategy(strategy_with_short,prices)